# Phase 1: Dataset Preparation & Environment Setup
Outdoor Object Detection & Face Recognition System

This notebook handles:
- Environment setup and dependency installation
- Dataset download (LFW, WiderFace, RTTS, BDD100K)
- Preprocessing (resize to 640x640, train/val/test split)
- Data augmentation (fog, rain, low-light, motion blur)
- Dataset statistics and verification

**Runtime**: GPU (T4) recommended for faster processing
**Storage**: Results saved to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
PROJECT_DIR = '/content/drive/MyDrive/computer_vision'
os.makedirs(PROJECT_DIR, exist_ok=True)
print(f"Project directory: {PROJECT_DIR}")

In [ ]:
%cd /content
!rm -rf computer_vision_expirement
!git clone https://github.com/Ib-Programmer/computer_vision_expirement.git
%cd computer_vision_expirement
!pip install -q -r requirements.txt
!pip install -q gdown

## 1.1 Dataset Download

In [ ]:
%cd /content/computer_vision_expirement
!python scripts/download_datasets.py

## 1.2 Preprocessing

In [ ]:
%cd /content/computer_vision_expirement
!python scripts/preprocess_data.py

## 1.3 Augmentation Preview (Visual Only)
Augmentations (fog, rain, low-light, motion blur, combined) are applied **on-the-fly during training** in Phase 3.
This saves ~35GB of disk space and gives better results (different random augmentation every epoch).

Below we generate a small visual preview to verify the augmentations look correct.

In [ ]:
%cd /content/computer_vision_expirement
!pip install -q albumentations

import cv2
import numpy as np
import matplotlib.pyplot as plt
import albumentations as A
import glob

# Define outdoor augmentations (same ones used in Phase 3 training)
augmentations = {
    'fog': A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.7, alpha_coef=0.08, p=1.0),
    'rain': A.RandomRain(slant_lower=-10, slant_upper=10, drop_length=20, drop_width=1,
                         drop_color=(200, 200, 200), blur_value=3, brightness_coefficient=0.7, p=1.0),
    'low_light': A.RandomBrightnessContrast(brightness_limit=(-0.5, -0.2), contrast_limit=(-0.3, 0.0), p=1.0),
    'motion_blur': A.MotionBlur(blur_limit=(7, 15), p=1.0),
    'combined': A.Compose([
        A.OneOf([
            A.RandomFog(fog_coef_lower=0.2, fog_coef_upper=0.5, alpha_coef=0.08, p=1.0),
            A.RandomRain(slant_lower=-10, slant_upper=10, drop_length=20, drop_width=1,
                         drop_color=(200, 200, 200), blur_value=3, brightness_coefficient=0.7, p=1.0),
        ], p=0.5),
        A.OneOf([
            A.RandomBrightnessContrast(brightness_limit=(-0.4, -0.1), contrast_limit=(-0.2, 0.0), p=1.0),
            A.MotionBlur(blur_limit=(7, 12), p=1.0),
        ], p=0.5),
    ]),
}

# Find sample images from preprocessed datasets
PROJECT_DIR = '/content/drive/MyDrive/computer_vision'
sample_paths = glob.glob(f'{PROJECT_DIR}/datasets/*_processed/train/*.jpg')[:3]
if not sample_paths:
    sample_paths = glob.glob(f'/content/computer_vision_expirement/datasets/*_processed/train/*.jpg')[:3]

if sample_paths:
    aug_names = list(augmentations.keys())
    fig, axes = plt.subplots(len(sample_paths), len(aug_names) + 1, figsize=(20, 4 * len(sample_paths)))
    if len(sample_paths) == 1:
        axes = [axes]

    for row, img_path in enumerate(sample_paths):
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        axes[row][0].imshow(img_rgb)
        axes[row][0].set_title('Original', fontsize=10)
        axes[row][0].axis('off')

        for col, aug_name in enumerate(aug_names, 1):
            transform = augmentations[aug_name]
            if isinstance(transform, A.Compose):
                aug_img = transform(image=img_rgb)['image']
            else:
                t = A.Compose([transform])
                aug_img = t(image=img_rgb)['image']
            axes[row][col].imshow(aug_img)
            axes[row][col].set_title(aug_name, fontsize=10)
            axes[row][col].axis('off')

    plt.suptitle('Augmentation Preview (applied on-the-fly during Phase 3 training)', fontsize=14)
    plt.tight_layout()
    plt.savefig(f'{PROJECT_DIR}/results/augmentation_preview.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("Preview saved. Full augmentations run on-the-fly in Phase 3.")
else:
    print("No preprocessed images found. Run preprocessing first.")

## 1.4 Dataset Statistics

In [ ]:
%cd /content/computer_vision_expirement
!python scripts/dataset_stats.py

## 1.5 Save to Google Drive (lightweight — results only)
Datasets stay on Colab's local disk (fast SSD). They get re-downloaded each session (~20 min).
Only small results (stats CSV, augmentation preview) are saved to Drive to preserve space.

In [ ]:
import os

PROJECT_DIR = '/content/drive/MyDrive/computer_vision'
RESULTS_DIR = f'{PROJECT_DIR}/results/phase1'
os.makedirs(RESULTS_DIR, exist_ok=True)

# Save only lightweight results to Drive (stats, preview image)
import shutil
import glob

# Copy stats CSV if generated
for f in glob.glob('/content/computer_vision_expirement/datasets/*stats*'):
    shutil.copy(f, RESULTS_DIR)

# Copy augmentation preview
preview = f'{PROJECT_DIR}/results/augmentation_preview.png'
if os.path.exists(preview):
    print(f"Augmentation preview saved: {preview}")

# Show what's on local disk (NOT copied to Drive)
import subprocess
local_size = subprocess.run(['du', '-sh', '/content/computer_vision_expirement/datasets'],
                            capture_output=True, text=True)
print(f"\nLocal datasets (Colab disk, NOT on Drive): {local_size.stdout.strip()}")

drive_size = subprocess.run(['du', '-sh', PROJECT_DIR], capture_output=True, text=True)
print(f"Drive usage: {drive_size.stdout.strip()}")

print(f"""
Phase 1 Complete!

Storage strategy:
  - Datasets stay on Colab local disk (re-download each session, ~20 min)
  - Only trained models + metrics saved to Drive (<1GB total)
  - Augmentations applied on-the-fly in Phase 3 (zero storage)
  - Free Drive (15GB) is more than enough!

Next: Open Phase2_Image_Enhancement.ipynb
""")

## Next Steps
- Open **Phase2_Image_Enhancement.ipynb** to evaluate enhancement models
- Each phase re-downloads datasets to Colab's fast local disk (~20 min)
- Only results (models, metrics) are saved to Google Drive